In [1]:
import sys
import os

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

In [3]:
from model.rec_model import microgrid

In [4]:
setup = {
  "horizon": 25,
  "timestep": 1,
  "add_unbalanced_module": True,
  "loss_load_cost": 10,
  "overgeneration_cost": 2,
  "components": [
    {
      "id": "3c03f396-98f2-46c7-945c-3c95408b6355",
      "type": "LoadModule",
      "params": {
        "time_series_profile": "house"
      }
    },
    {
      "id": "32e548e7-8b69-49b6-b471-54908cd211d3",
      "type": "RenewableModule",
      "params": {
        "time_series_profile": "solar_1"
      }
    },
    {
      "id": "ba34e364-9c4a-412f-9b3c-9068335d15d3",
      "type": "BatteryModule",
      "params": {
        "min_capacity": 0,
        "max_capacity": 50,
        "max_charge": 10,
        "max_discharge": 10,
        "efficiency": 0.95,
        "battery_cost_cycle": 0,
        "init_soc": 0.5
      }
    },
    {
      "id": "ba34e364-9c4a-412f-9b3c-9068335d15d4",
      "type": "BatteryModule",
      "params": {
        "min_capacity": 0,
        "max_capacity": 150,
        "max_charge": 10,
        "max_discharge": 10,
        "efficiency": 0.95,
        "battery_cost_cycle": 0,
        "init_soc": 0.5
      }
    },
    {
      "id": "4ba2cfa4-7323-4f0a-ac7f-fa73f1177e49",
      "type": "GridModule",
      "params": {
        "max_import": 100,
        "max_export": 100,
        "cost_per_unit_co2": 0,
        "time_series_profile": "grid_1"
      }
    }
  ]
}

In [7]:
microgrid.setup(setup)

In [9]:
microgrid.microgrid.modules.to_tuples()

[('load',
  LoadModule(time_series=<class 'numpy.ndarray'>, forecaster=NoForecaster, forecast_horizon=0, forecaster_increase_uncertainty=False, raise_errors=False)),
 ('renewable',
  RenewableModule(time_series=<class 'numpy.ndarray'>, raise_errors=False, forecaster=NoForecaster, forecast_horizon=0, forecaster_increase_uncertainty=False, provided_energy_name=renewable_used)),
 ('balancing',
  UnbalancedEnergyModule(raise_errors=False, loss_load_cost=10, overgeneration_cost=2)),
 ('battery',
  BatteryModule(min_capacity=0, max_capacity=50, max_charge=10, max_discharge=10, efficiency=0.95, battery_cost_cycle=0, battery_transition_model=None, init_charge=None, init_soc=0.5, raise_errors=False)),
 ('battery',
  BatteryModule(min_capacity=0, max_capacity=150, max_charge=10, max_discharge=10, efficiency=0.95, battery_cost_cycle=0, battery_transition_model=None, init_charge=None, init_soc=0.5, raise_errors=False)),
 ('grid', GridModule(max_import=100, max_export=100))]

In [11]:
from pymgrid.envs import ContinuousMicrogridEnv

In [13]:
env = ContinuousMicrogridEnv(microgrid.microgrid)

ValueError: need at least one array to concatenate

In [33]:
from gym.spaces import Box, Dict, Tuple, flatten_space

In [41]:
nested_action_space = Dict({name: Tuple([module.action_space['normalized'] for module in modules_list])
                                 for name, modules_list in microgrid.microgrid.modules.controllable.iterdict()})
nested_action_space

Dict('battery': Tuple(Box(0.0, 1.0, (1,), float64), Box(0.0, 1.0, (1,), float64)), 'grid': Tuple(Box(0.0, 1.0, (1,), float64)))

In [45]:
flatten_space(nested_action_space)

Box(0.0, 1.0, (3,), float64)

In [29]:
env = ContinuousMicrogridEnv(microgrid.microgrid.modules.to_tuples())

ValueError: need at least one array to concatenate